# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data.dtypes

CustomerID      int64
FirstName      object
LastName       object
SalesID         int64
ProductID       int64
ProductName    object
Quantity        int64
dtype: object

In [5]:
customer_products = data.groupby(by=['CustomerID', 'ProductName'], as_index=False).agg({'Quantity': 'sum'})
customer_products

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the `products`, the columns should represent the `customers`, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
pivot = pd.pivot_table(data=customer_products, values='Quantity', index='ProductName', columns='CustomerID').fillna(0)
pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
distances = pd.DataFrame(1/(1+squareform(pdist(pivot.T, metric='euclidean'))),
                        index=pivot.columns,
                        columns=pivot.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
custID = 33

# list of the top 5 most similar customers for a specific CustomerID
similar_customers_top5 = list(distances[custID].sort_values(ascending=False)[1:][:5].index)
similar_customers_top5

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_customers_top5)]
sim_cust_prod

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
ranked_products = sim_cust_prod.groupby(by='ProductName', as_index=False).agg({'Quantity':'sum'}) \
                            .sort_values(by='Quantity', ascending=False)
ranked_products

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3
...,...,...
85,Hinge W Undercut,1
86,Ice Cream Bar - Hageen Daz To,1
87,Jagermeister,1
88,Jolt Cola - Electric Blue,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
custID

33

In [12]:
# List of products (Starswith A) which customerID 33 purchased 
custID_33_purchaed = customer_products.loc[customer_products['CustomerID'] == 33,:]
custID_33_purchaed

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
5,33,"Beef - Chuck, Boneless",1
6,33,Beef - Prime Rib Aaa,1
7,33,Beer - Original Organic Lager,1
8,33,Beer - Rickards Red,1
9,33,Black Currants,1


In [13]:
# custID = 33
pd.DataFrame(pivot[custID])

,33
ProductName,
Anchovy Paste - 56 G Tube,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0
Appetizer - Mushroom Tart,0.0
Appetizer - Sausage Rolls,0.0
Apricots - Dried,1.0
...,...
Yeast Dry - Fermipan,0.0
Yoghurt Tubes,0.0
"Yogurt - Blueberry, 175 Gr",0.0


In [14]:
df_merged = pd.merge(ranked_products, pd.DataFrame(pivot[custID]), on='ProductName' )
df_merged

,ProductName,Quantity,33
0,Butter - Unsalted,3,0.0
1,Wine - Ej Gallo Sierra Valley,3,0.0
2,Towels - Paper / Kraft,3,1.0
3,Soup - Campbells Bean Medley,3,0.0
4,Wine - Blue Nun Qualitatswein,3,0.0
...,...,...,...
213,Hinge W Undercut,1,0.0
214,Ice Cream Bar - Hageen Daz To,1,0.0
215,Jagermeister,1,0.0
216,Jolt Cola - Electric Blue,1,0.0


In [15]:
# rename column names
df_merged.columns = ['ProductName', 'Quantity', 'Purchased']
df_merged

,ProductName,Quantity,Purchased
0,Butter - Unsalted,3,0.0
1,Wine - Ej Gallo Sierra Valley,3,0.0
2,Towels - Paper / Kraft,3,1.0
3,Soup - Campbells Bean Medley,3,0.0
4,Wine - Blue Nun Qualitatswein,3,0.0
...,...,...,...
213,Hinge W Undercut,1,0.0
214,Ice Cream Bar - Hageen Daz To,1,0.0
215,Jagermeister,1,0.0
216,Jolt Cola - Electric Blue,1,0.0


In [16]:
rec_top5_product = df_merged[df_merged['Purchased']==0][:5]
rec_top5_product

,ProductName,Quantity,Purchased
0,Butter - Unsalted,3,0.0
1,Wine - Ej Gallo Sierra Valley,3,0.0
3,Soup - Campbells Bean Medley,3,0.0
4,Wine - Blue Nun Qualitatswein,3,0.0
6,Chicken - Soup Base,2,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [17]:
# Creating an empty dictionary that will hold the recommendations for all customers.
recommendations = {}

# Creating a list of unique CustomerIDs to iterate over.
customers = list(data['CustomerID'].unique())

for customer in customers:
    similar_customers_top5 = list(distances[customer].sort_values(ascending=False)[1:][:5].index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_customers_top5)]
    ranked_products = sim_cust_prod.groupby(by='ProductName', as_index=False).agg({'Quantity':'sum'}) \
                                   .sort_values(by='Quantity', ascending=False)
    df_merged = pd.merge(ranked_products, pd.DataFrame(pivot[customer]), on='ProductName' )
    df_merged.columns = ['ProductName', 'Quantity', 'Purchased']
    rec_top5_product = df_merged[df_merged['Purchased']==0][:5]
    
    recommendations[customer] = list(rec_top5_product['ProductName'])


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [22]:
#pd.DataFrame.from_dict(recommendations, orient='index')
df_recommendation = pd.DataFrame(recommendations).T.reset_index()
df_recommendation.rename(columns={'index':'CustomerID',
                                   0:'rec_product_1',
                                   1:'rec_product_2',
                                   2:'rec_product_3',
                                   3:'rec_product_4',
                                   4:'rec_product_5'},
                                   inplace=True)
df_recommendation = df_recommendation.sort_values(by='CustomerID').reset_index(drop=True)
df_recommendation

,CustomerID,rec_product_1,rec_product_2,rec_product_3,rec_product_4,rec_product_5
0,33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
1,200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
2,264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice","Fish - Scallops, Cold Smoked"
3,356,"Veal - Inside, Choice",Lamb - Ground,Pomello,Butter - Unsalted,"Tart Shells - Sweet, 4"
4,412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Soup - Campbells Bean Medley
...,...,...,...,...,...,...
995,97928,"Soup - Campbells, Lentil","Cheese - Brie,danish",Wiberg Super Cure,Pants Custom Dry Clean,Shrimp - 31/40
996,98069,Skirt - 29 Foot,Beans - Kidney White,Milk - 1%,Cheese - Taleggio D.o.p.,Sprouts - Baby Pea Tendrils
997,98159,"Lamb - Whole, Fresh",Bread - Raisin Walnut Oval,Chips Potato Salt Vinegar 43g,Tea - English Breakfast,Wine - Redchard Merritt
998,98185,Crackers - Trio,Pernod,Tea - Jasmin Green,Pastry - Choclate Baked,"Peas - Pigeon, Dry"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [23]:
def recommend_product (dataframe, metric):

    customer_products = dataframe.groupby(by=['CustomerID', 'ProductName'], as_index=False).agg({'Quantity': 'sum'})
    pivot = pd.pivot_table(data=customer_products, values='Quantity', index='ProductName', columns='CustomerID').fillna(0)
    distances = pd.DataFrame(1/(1+squareform(pdist(pivot.T, metric=metric))), index=pivot.columns,columns=pivot.columns)
    
    
    recommendations = {}
    customers = list(data['CustomerID'].unique())      
    
    for customer in customers:
        
        similar_customers_top5 = list(distances[customer].sort_values(ascending=False)[1:][:5].index)
        sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_customers_top5)]
        ranked_products = sim_cust_prod.groupby(by='ProductName', as_index=False).agg({'Quantity':'sum'}) \
                                   .sort_values(by='Quantity', ascending=False)
        df_merged = pd.merge(ranked_products, pd.DataFrame(pivot[custID]), on='ProductName' )
        df_merged.columns = ['ProductName', 'Quantity', 'Purchased']
        rec_top5_product = df_merged[df_merged['Purchased']==0][:5]
        recommendations[customer] = list(rec_top5_product['ProductName'])
    
    df_recommendation = pd.DataFrame(recommendations).T.reset_index()
    df_recommendation.rename(columns={'index':'CustomerID',
                                   0:'rec_product_1',
                                   1:'rec_product_2',
                                   2:'rec_product_3',
                                   3:'rec_product_4',
                                   4:'rec_product_5'},
                                   inplace=True)
    df_recommendation = df_recommendation.sort_values(by='CustomerID').reset_index(drop=True)
    
    return df_recommendation

In [24]:
recommend_product (data,'euclidean')

,CustomerID,rec_product_1,rec_product_2,rec_product_3,rec_product_4,rec_product_5
0,33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
1,200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Lamb - Ground
2,264,Wine - Blue Nun Qualitatswein,"Fish - Scallops, Cold Smoked",Cheese - Cambozola,Table Cloth 54x72 White,Cheese Cloth No 100
3,356,Wine - Ej Gallo Sierra Valley,Lamb - Ground,Wine - Blue Nun Qualitatswein,Pomello,Butter - Unsalted
4,412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,Soup - Campbells Bean Medley,"Sauce - Gravy, Au Jus, Mix"
...,...,...,...,...,...,...
995,97928,Bouq All Italian - Primerba,Tea - Jasmin Green,"Soup - Campbells, Lentil",Arizona - Green Tea,"Cheese - Brie,danish"
996,98069,Skirt - 29 Foot,Chocolate - Dark,Beans - Kidney White,Milk - 1%,Longos - Grilled Salmon With Bbq
997,98159,Chips Potato All Dressed - 43g,Table Cloth 81x81 White,"Lamb - Whole, Fresh",Pernod,"Ice - Clear, 300 Lb For Carving"
998,98185,Halibut - Steaks,Cod - Black Whole Fillet,Wine - Pinot Noir Latour,Chicken - Wieners,Crackers - Trio


In [25]:
recommend_product (data,'correlation')

,CustomerID,rec_product_1,rec_product_2,rec_product_3,rec_product_4,rec_product_5
0,33,Knife Plastic - White,Muffin - Zero Transfat,Banana Turning,Crush - Cream Soda,Veal - Osso Bucco
1,200,Olives - Kalamata,Pop Shoppe Cream Soda,Otomegusa Dashi Konbu,Water - Spring Water 500ml,Lamb - Ground
2,264,"Lemonade - Natural, 591 Ml","Water - Mineral, Natural",Wine - Toasted Head,Wine - Blue Nun Qualitatswein,Muffin Mix - Blueberry
3,356,Tomatoes Tear Drop,Scallops - Live In Shell,Scallops - 10/20,Cheese - Taleggio D.o.p.,"Coconut - Shredded, Sweet"
4,412,Banana - Leaves,Pernod,Cake - Mini Cheesecake,Butter - Unsalted,"Salmon - Atlantic, Skin On"
...,...,...,...,...,...,...
995,97928,Tea - Jasmin Green,Bouq All Italian - Primerba,Bread - French Baquette,Chocolate - Dark,Lettuce - Treviso
996,98069,Mustard Prepared,Hot Chocolate - Individual,Longos - Grilled Salmon With Bbq,Flour - Pastry,"Chocolate - Semi Sweet, Calets"
997,98159,Crush - Cream Soda,Table Cloth 81x81 White,Yoghurt Tubes,"Liners - Banana, Paper","Water, Tap"
998,98185,Cheese - Taleggio D.o.p.,Squid U5 - Thailand,Cocktail Napkin Blue,Mussels - Cultivated,"Peas - Pigeon, Dry"


In [26]:
recommend_product (data,'cityblock')

,CustomerID,rec_product_1,rec_product_2,rec_product_3,rec_product_4,rec_product_5
0,33,Chocolate - Dark,Wine - Charddonnay Errazuriz,Bay Leaf,"Oranges - Navel, 72",Wine - Ej Gallo Sierra Valley
1,200,Soup - Campbells Bean Medley,Lamb - Ground,Pomello,Bay Leaf,Pork - Kidney
2,264,Muffin - Carrot Individual Wrap,"Tart Shells - Sweet, 4",Pork - Kidney,Durian Fruit,Vanilla Beans
3,356,Butter - Unsalted,Wine - Blue Nun Qualitatswein,Table Cloth 81x81 White,Bar - Granola Trail Mix Fruit Nut,"Beets - Candy Cane, Organic"
4,412,Wine - Blue Nun Qualitatswein,Butter - Unsalted,Lamb - Ground,Table Cloth 81x81 White,Soup - Campbells Bean Medley
...,...,...,...,...,...,...
995,97928,Appetizer - Sausage Rolls,Wine - Chardonnay South,Bacardi Breezer - Tropical,Soup - Campbells Bean Medley,"Nut - Chestnuts, Whole"
996,98069,Scallop - St. Jaques,Wine - Chardonnay South,Knife Plastic - White,Oil - Shortening - All - Purpose,Flour - Pastry
997,98159,Butter - Unsalted,Knife Plastic - White,Soup - Campbells Bean Medley,Cod - Black Whole Fillet,"Wine - Magnotta, Merlot Sr Vqa"
998,98185,Knife Plastic - White,Tia Maria,Berry Brulee,Muffin - Carrot Individual Wrap,Pomello


In [30]:
recommend_product (data,'cosine')

,CustomerID,rec_product_1,rec_product_2,rec_product_3,rec_product_4,rec_product_5
0,33,Knife Plastic - White,"Soup - Campbells, Beef Barley",Onions - Cippolini,Tea - Herbal Sweet Dreams,Muffin - Zero Transfat
1,200,Olives - Kalamata,Mustard Prepared,Pop Shoppe Cream Soda,Longos - Grilled Salmon With Bbq,"Chocolate - Semi Sweet, Calets"
2,264,"Lemonade - Natural, 591 Ml",Pickerel - Fillets,Muffin Mix - Blueberry,Wine - Blue Nun Qualitatswein,"Water - Mineral, Natural"
3,356,Tomatoes Tear Drop,Sobe - Tropical Energy,Bread - English Muffin,"Thyme - Lemon, Fresh",Olive - Spread Tapenade
4,412,Banana - Leaves,Beans - Wax,Durian Fruit,Whmis - Spray Bottle Trigger,Wine - Hardys Bankside Shiraz
...,...,...,...,...,...,...
995,97928,Apricots - Halves,Cheese - Mozzarella,Tofu - Firm,"Spoon - Soup, Plastic",Beef - Montreal Smoked Brisket
996,98069,Mustard Prepared,Longos - Grilled Salmon With Bbq,Sprouts - Baby Pea Tendrils,"Chocolate - Semi Sweet, Calets",Juice - Orange
997,98159,Crush - Cream Soda,Table Cloth 81x81 White,Yoghurt Tubes,"Liners - Banana, Paper","Water, Tap"
998,98185,"Peas - Pigeon, Dry",Halibut - Fletches,Chicken - Wieners,Wasabi Powder,Squid U5 - Thailand


In [29]:
recommend_product (data,'jaccard')

,CustomerID,rec_product_1,rec_product_2,rec_product_3,rec_product_4,rec_product_5
0,33,Loquat,Milk - 2%,Muffin Mix - Blueberry,Blueberries,"Cheese - Brie, Triple Creme"
1,200,Muffin - Carrot Individual Wrap,Pomello,Berry Brulee,Table Cloth 81x81 White,Pork - Kidney
2,264,Flavouring - Orange,Chocolate - Compound Coating,Beer - Blue,Ice Cream Bar - Hageen Daz To,Bread - Italian Corn Meal Poly
3,356,Corn Meal,Wine - Ej Gallo Sierra Valley,"Tart Shells - Sweet, 4",Beef - Inside Round,Sardines
4,412,Wine - Wyndham Estate Bin 777,Cumin - Whole,Milk - 2%,Watercress,Curry Paste - Madras
...,...,...,...,...,...,...
995,97928,Tray - 16in Rnd Blk,Pernod,Pastry - Choclate Baked,"Wine - White, Mosel Gold",Cocoa Butter
996,98069,Spinach - Baby,Beef - Ground Medium,Tea - Jasmin Green,Bread - French Baquette,Cheese - Parmesan Cubes
997,98159,Chocolate - Compound Coating,Apricots - Halves,Oil - Shortening - All - Purpose,"Wine - White, Mosel Gold",Mustard Prepared
998,98185,Coffee - Irish Cream,Soup V8 Roasted Red Pepper,Juice - Apple Cider,"Cheese - Brie,danish",Whmis - Spray Bottle Trigger


---